### **Importing python Libraries**

In [0]:
import pandas as pd
import numpy as np


### Loading csv file as dataframe

In [0]:
parking_path = "/Volumes/parking_catalog/parking_schema/parking_volume/Parking meter locations.csv"

df_parking = pd.read_csv(parking_path)
df_parking.head()

,index,zone,area,sub_area,pole,config_code,config_name,longitude,latitude
0,0,City,Barrio Logan,2900 ADDISON ST,ADN-2912,9116,30 Min Max $1.25 HR 8am-6pm Mon-Sat,-117.230904,32.721670
1,1,City,Barrio Logan,2900 ADDISON ST,ADN-2914,9116,30 Min Max $1.25 HR 8am-6pm Mon-Sat,-117.230913,32.721575
2,2,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1003,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145178,32.700353
3,3,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1005,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145178,32.700352
4,4,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1011,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145349,32.700155


### Creating dummy rate from poles and zones in parking table

In [0]:
def assign_rate(zone):
    '''
         assign rate based on zone in parking table
    '''
    if zone == "Downtown":
        return np.random.choice([7,8,9]) 
    elif zone == "Mid-City":
        return np.random.choice([5,6,7]) 
    elif zone == "Uptown":
        return np.random.choice([3,4,5]) 
    else:
        return np.random.choice([4,5,6])

rate_df = pd.DataFrame()
rate_df['pole'] = df_parking['pole']
rate_df['hourly_rate'] = df_parking['zone'].apply(assign_rate)
rate_df['rate_type'] = np.random.choice(['weekday','weekend'], size=len(rate_df))

In [0]:
# storing the ratedf as csv in volume
rate_csv_path = "/Volumes/parking_catalog/parking_schema/parking_volume/rate.csv"
rate_df.to_csv(rate_csv_path, index=False)
print(f"Rate CSV saved at: {rate_csv_path}")

Rate CSV saved at: /Volumes/parking_catalog/parking_schema/parking_volume/rate.csv


###  **Turning rate csv into rate managed **table****

In [0]:
rate_df=spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/Volumes/parking_catalog/parking_schema/parking_volume/rate.csv")

In [0]:
rate_df.createOrReplaceTempView("rate_view")

spark.sql('''
          CREATE OR REPLACE TABLE parking_catalog.parking_schema.rate_table
          AS
          SELECT * FROM rate_view''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

### Deleting few poles for left join with parking table

In [0]:
%sql
delete from parking_catalog.parking_schema.rate_table where pole like 'ADN%';

num_affected_rows
2


### Inserting rows with poles that does not exit in parking for right join

In [0]:
%sql
INSERT INTO parking_catalog.parking_schema.rate_table
VALUES ('9999', 5, 'weekday');

INSERT INTO parking_catalog.parking_schema.rate_table
VALUES ('10000', 6, 'weekend');

INSERT INTO parking_catalog.parking_schema.rate_table
VALUES ('10001', 4, 'weekday');


num_affected_rows,num_inserted_rows
1,1
